<a href="https://colab.research.google.com/github/Npoitier/Implementez_un_modele_de_scoring/blob/main/P7_choix_du_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span id="rt" style="font-family:Latin fonts;font-size:1.8em;">Sommaire</span>
0. [Ressources Projet](#s0) <br>
    0.1. [Imports](#s0_1) <br>
    0.2. [Constantes](#s0_2) <br>
    0.3. [Fonctions](#s0_3) <br>
    0.4. [Feature Engineering](#s0_4) <br>
1. [Données déséquilibrées](#s1) <br>
    1.1. [Undersampling / Tomek Links](#s1_1) <br>
    1.2. [Oversampling / Smote](#s1_2) <br>
    1.3. [Métriques](#s1_3) <br>
2. [Classification supervisée](#s2) <br>
3. [Fixer les hypers-paramètres](#s3) <br>
4. [Choix de l'algorythme](#s4) <br>

# <span id="s0" style="font-family:Latin fonts;font-size:1.8em;"> 0. Ressources Projet</span><br>
[Sommaire](#rt)

## <span id="s0_1" style="font-family:Latin fonts;font-size:1.6em;">Imports</span><br>
[Sommaire](#rt)

In [6]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import train_test_split

#from sklearn.pipeline import make_pipeline
from imblearn.under_sampling import TomekLinks
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display='diagram') 
# from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline, Pipeline 
from sklearn.preprocessing import FunctionTransformer

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## <span id="s0_2" style="font-family:Latin fonts;font-size:1.6em;">Constantes</span><br>
[Sommaire](#rt)

In [8]:
chemin = 'drive/My Drive/Colab Notebooks/input/'
chemin_save = 'drive/My Drive/Colab Notebooks/'

## <span id="s0_3" style="font-family:Latin fonts;font-size:1.6em;">Fonctions</span><br>
[Sommaire](#rt)

In [ ]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [9]:
def df_list_colonnes(df):
  df = df.convert_dtypes()
  categorical_columns = [col for col in df.columns if df[col].dtype == 'string']
  discret_columns = [col for col in df.columns if df[col].dtype == 'Int64']
  float_columns = [col for col in df.columns if df[col].dtype == 'Float64']
  return categorical_columns, discret_columns, float_columns

## <span id="s0_4" style="font-family:Latin fonts;font-size:1.6em;">Feature Engineering</span><br>

On reprend intégralement le code proposé sur le notebook [https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script](https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script) <br>
On va juste différer les encodages catégoriels des catégories au niveau du client (application_train_test) pour l'application de Smote. <br>
Les catégories des fichiers autres que client, sont utilisés comme des regroupements et ne sont donc pas exclusifs au niveau du client <br>
[Sommaire](#rt)

In [ ]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))
    
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv(chemin+'application_train.csv', nrows= num_rows)
    test_df = pd.read_csv(chemin+'application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    # for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    #    df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    # df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv(chemin+'bureau.csv', nrows = num_rows)
    bb = pd.read_csv(chemin+'bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv(chemin+'previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv(chemin+'POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg
    
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv(chemin+'installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv(chemin+'credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

    

def main(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()    
    return df


In [ ]:
df = main()
# traitement des valeurs infini
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# re-typage des colonnes
df = df.convert_dtypes()
# sauvegarde
df.to_csv(chemin_save+"df_final.csv")


Train samples: 307511, test samples: 48744
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 40s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 30s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 17s
Installments payments df shape: (339587, 26)
Process installments payments - done in 41s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 28s


In [ ]:
df.to_csv(chemin_save+"df_final.csv")

In [ ]:
listype = df.dtypes.unique().tolist()
listetypename = []
for i in listype:
    print(i.name)
    listetypename.append(i.name)
print(df.dtypes.unique().tolist())

Int64
string
Float64
[Int64Dtype(), string[python], Float64Dtype()]


In [ ]:
df = pd.read_csv(chemin_save+"df_final.csv",index_col=0)

In [ ]:
df_target_null = df.loc[df["TARGET"].isnull()]
df_target_null.to_csv(chemin_save+"df_null.csv")
df_target_0 = df.loc[df["TARGET"] == 0]
df_target_0.to_csv(chemin_save+"df_0.csv")
df_target_1 = df.loc[df["TARGET"] == 1]
df_target_1.to_csv(chemin_save+"df_1.csv")
print('total',df.shape[0])
print('nule',df_target_null.shape[0])
non_null = df_target_0.shape[0] + df_target_1.shape[0]
print('non nule',non_null,'vérif',df.shape[0]-df_target_null.shape[0])
pourcent_0 = df_target_0.shape[0] / (df_target_0.shape[0]+df_target_1.shape[0])
pourcent_1 = df_target_1.shape[0] / (df_target_0.shape[0]+df_target_1.shape[0])
print('valeur 0',df_target_0.shape[0],pourcent_0,'%')
print('valeur 1',df_target_1.shape[0],pourcent_1,'%')

total 356251
nule 48744
non nule 307507 vérif 307507
valeur 0 282682 0.9192701304360551 %
valeur 1 24825 0.08072986956394489 %


In [ ]:
# rechargement
df_target_null = pd.read_csv(chemin_save+"df_null.csv",index_col=0)
df_target_0 = pd.read_csv(chemin_save+"df_0.csv",index_col=0)
df_target_1 = pd.read_csv(chemin_save+"df_1.csv",index_col=0)
pourcent_0 = df_target_0.shape[0] / (df_target_0.shape[0]+df_target_1.shape[0])
pourcent_1 = df_target_1.shape[0] / (df_target_0.shape[0]+df_target_1.shape[0])


In [ ]:
# établissement des listes de colonnes
categorical_columns, discret_columns, float_columns = df_list_colonnes(df_target_null)
print(len(categorical_columns))
print(len(discret_columns))
print(len(float_columns))
print(len(df_target_null.columns))

16
194
468
678


# <span id="s1" style="font-family:Latin fonts;font-size:1.8em;"> 1. Données déséquilibrées</span><br>
[https://www.youtube.com/watch?v=R5YVj-_d5AI&ab_channel=LarevueIA](https://www.youtube.com/watch?v=R5YVj-_d5AI&ab_channel=LarevueIA)<br>
[https://www.youtube.com/watch?v=EheO1scMXkQ&ab_channel=MASTER2SISEDATASCIENCE](https://www.youtube.com/watch?v=EheO1scMXkQ&ab_channel=MASTER2SISEDATASCIENCE) <br>
[https://mlr.mlr-org.com/articles/tutorial/over_and_undersampling.html](https://mlr.mlr-org.com/articles/tutorial/over_and_undersampling.html)<br>
[Sommaire](#rt)

In [ ]:
# on enleve TARGET de la liste des colonnes
y_cols = ['TARGET']
X_cols = [col for col in df_target_0.columns.tolist() if col != 'TARGET']

# on sépare
y = df_target_0[y_cols]
X = df_target_0[X_cols]
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, test_size=0.2, random_state=0)

y = df_target_1[y_cols]
X = df_target_1[X_cols]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = X_train0.append(X_train1)
print('X_train',X_train.shape[0])
X_test = X_test0.append(X_test1)
print('X_test',X_test.shape[0])
y_train = y_train0.append(y_train1)
print('y_train',y_train.shape[0])
y_test = y_test0.append(y_test1)
print('y_test',y_test.shape[0])

X_train 246005
X_test 61502
y_train 246005
y_test 61502


In [ ]:
# save
X_train.to_csv(chemin_save+"X_train.csv")
X_test.to_csv(chemin_save+"X_test.csv")
y_train.to_csv(chemin_save+"y_train.csv")
y_test .to_csv(chemin_save+"y_test .csv")


In [ ]:
X_train = pd.read_csv(chemin_save+"X_train.csv",index_col=0)
# X_test = pd.read_csv(chemin_save+"X_test.csv",index_col=0)
y_train = pd.read_csv(chemin_save+"y_train.csv",index_col=0)
# y_test  = pd.read_csv(chemin_save+"y_test .csv",index_col=0)


In [12]:
# on triche pour des problèmes de mémoire
X_train =  pd.read_csv(chemin_save+"X_test.csv",index_col=0)
y_train = pd.read_csv(chemin_save+"y_test .csv",index_col=0)

## <span id="s1_1" style="font-family:Latin fonts;font-size:1.6em;">Undersampling / Tomek Links</span><br>
Le sous-échantillonnage (undersampling) consiste à rééquilibrer le jeu de données en diminuant le nombre d’instances de la classe majoritaire.<br>
Dans le cas de Tomek Links, il s'agit de supprimer les points les plus proches de la classe minoritaire. <br>
[Sommaire](#rt)

In [13]:
# on sépare données catégorielles des données numériques
X_train_categorical_columns, X_train_discret_columns, X_train_float_columns = df_list_colonnes(X_train)
X_train_numeric_colonnes = X_train_discret_columns + X_train_float_columns
print(len(X_train_categorical_columns))
print(len(X_train_numeric_colonnes))

16
661


In [10]:
# fonction TomekLinks
def tl_df(df):
  # dans le pipeline, les colonnes sont permutées
  df_num = df[:,range(len(X_train_numeric_colonnes))]
  df_categ = df[:,range(len(num_col),len(X_train_categorical_columns+X_train_numeric_colonnes))]
  Tomek_Sampling = TomekLinks()
  X_res, y_res = Tomek_Sampling.fit_resample(df_num, y_train)
  idx = Tomek_Sampling.sample_indices_

  X_res = pd.DataFrame(X_res,columns= X_train_numeric_colonnes)
  X_res.index = idx

  X_cat = pd.DataFrame(df_categ,columns= X_train_categorical_columns)
  X_cat.index = X_train.index

  X_res = X_res.join(X_cat)
  return X_res
under_fonction = FunctionTransformer(tl_df)

In [14]:
# construction du pipeline TomekLinks




transfo_quanti = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='median')),
    ('standard', RobustScaler()) 
    ])

transfo_quali = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='constant', fill_value='manquant')) #,
    #('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

preparation = ColumnTransformer(
    transformers=[
        ('quanti', transfo_quanti , X_train_numeric_colonnes),
        ('quali', transfo_quali , X_train_categorical_columns)
        ])
encode_cat = ColumnTransformer(
    transformers=[
        ('quanti',OneHotEncoder(handle_unknown='ignore'),X_train_categorical_columns )
        ], remainder ='passthrough')



TL_pipe = Pipeline(
    steps=[
        ('preparation',preparation) ,
        ('under', under_fonction),
        ('encode', encode_cat)
    ]
)



In [21]:
TL_pipe

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('quanti',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standard',
                                                                   RobustScaler())]),
                                                  ['index', 'SK_ID_CURR',
                                                   'CNT_CHILDREN', 'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGISTRATION',
                                                   'DAYS_ID_PUBLISH',
                                                   'OWN_CAR_AGE', 'FLAG_MOBIL',
                                                   'FLAG_EMP_PHONE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_CONT_MOBILE...
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['NAME_CONTRACT_TYPE',
                                                   'CODE_GENDER',
                                                   'FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_TYPE_SUITE',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'WEEKDAY_APPR_PROCESS_START',
                                                   'ORGANIZATION_TYPE',
                                                   'FONDKAPREMONT_MODE',
                                                   'HOUSETYPE_MODE',
                                                   'WALLSMATERIAL_MODE',
                                                   'EMERGENCYSTATE_MODE'])]))])

In [ ]:
TL_transfo = TL_pipe.fit(X_train)

In [ ]:
df = TL_transfo.transform(X_train)



In [ ]:
print(df.shape)
df

(59557, 818)


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
list_cols = pd.Series(TL_transfo['encode'].get_feature_names_out().tolist())
# print(list_cols)
list_cols = list_cols.str.replace('quanti__','').str.replace('remainder__','')
list_cols = list_cols.tolist()
print(list_cols)

['NAME_CONTRACT_TYPE_Cash loans', 'NAME_CONTRACT_TYPE_Revolving loans', 'NAME_CONTRACT_TYPE_nan', 'CODE_GENDER_F', 'CODE_GENDER_M', 'CODE_GENDER_nan', 'FLAG_OWN_CAR_N', 'FLAG_OWN_CAR_Y', 'FLAG_OWN_CAR_nan', 'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y', 'FLAG_OWN_REALTY_nan', 'NAME_TYPE_SUITE_Children', 'NAME_TYPE_SUITE_Family', 'NAME_TYPE_SUITE_Group of people', 'NAME_TYPE_SUITE_Other_A', 'NAME_TYPE_SUITE_Other_B', 'NAME_TYPE_SUITE_Spouse, partner', 'NAME_TYPE_SUITE_Unaccompanied', 'NAME_TYPE_SUITE_manquant', 'NAME_TYPE_SUITE_nan', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Working', 'NAME_INCOME_TYPE_nan', 'NAME_EDUCATION_TYPE_Academic degree', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher', 'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_EDUCATION_TYPE_nan', 'NAME_FAMIL

In [ ]:


# on sépare les colonnes catégorielles des autres
X_train_categ = X_train[X_train_categorical_columns]
X_train_num = X_train[X_train_other_columns]
X_test_num = X_test[X_train_other_columns]

# on impute la médiane du train aux valeurs manquantes
impute = SimpleImputer(strategy='median')
X_train_num = impute.fit_transform(X_train_num)
X_test_num = impute.fit_transform(X_test_num)


C:\Users\nisae\AppData\Local\Temp\ipykernel_6588\2476619508.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_num.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
stand = RobustScaler()
# on entraine RobustScaler sur les données numériques du train
stand.fit(X_train_num)
# on remplace les données numérique du train
X_train_num = stand.transform(X_train_num)
# on remplace les données numérique du test à partir du modèle entrainé sur le train
# X_test[listnum] = stand.transform(X_test[listnum])

In [ ]:
# construction de la pipeline

Tomek_Sampling = TomekLinks()
X_res, y_res = Tomek_Sampling.fit_resample(X_train_num, y_train)
print(y_res['TARGET'].value_counts())

0.0    219090
1.0     19860
Name: TARGET, dtype: int64


In [ ]:
print(y_train['TARGET'].value_counts())

0.0    226145
1.0     19860
Name: TARGET, dtype: int64


In [ ]:
# encodage des données catégorielles
enc = OneHotEncoder()
enc = enc.fit(df[listobject])
test = enc.transform(df[listobject]).toarray() 
Encode_df = pd.DataFrame(test, columns =enc.get_feature_names_out())
Encode_df.shape

## <span id="s1_2" style="font-family:Latin fonts;font-size:1.6em;">Oversampling / Smote</span><br>
Le sur-échantillonnage (oversampling) consiste à rééquilibrer le jeu de données en augmentant artificiellement le nombre d’instances de la classe minoritaire. <br>
[Sommaire](#rt)

In [5]:
# on triche pour des problèmes de mémoire
X_train =  pd.read_csv(chemin_save+"X_test.csv",index_col=0)
y_train = pd.read_csv(chemin_save+"y_test .csv",index_col=0)

# on sépare données catégorielles des données numériques
X_train_categorical_columns, X_train_discret_columns, X_train_float_columns = df_list_colonnes(X_train)
X_train_numeric_colonnes = X_train_discret_columns + X_train_float_columns
print(len(X_train_categorical_columns))
print(len(X_train_numeric_colonnes))


16
661


In [15]:
# Importation du package
from imblearn.over_sampling import SMOTENC

# Définition de l'instance SMOTE où 1 désigne la colonne catégorielle
# sm = SMOTENC(categorical_features=X_train_categorical_columns, sampling_strategy=0.1, k_neighbors=2)
sm = SMOTENC(categorical_features=[i for i in range(661,677)], sampling_strategy=0.1, k_neighbors=2)
# Application du SMOTE aux données
#X_res_SMOTENC, y_res_SMOTENC = sm.fit_resample(X_train, y_train)

In [16]:
def SN_to_df(df):
  new_liste_cols = X_train_numeric_colonnes + X_train_categorical_columns
  df = pd.DataFrame(df,columns= new_liste_cols)
  return df
SN_to_df_step = FunctionTransformer(SN_to_df)

In [59]:
discrete_values = [1.3,25.6,3.6]
values_to_discretize = [5,6,8,5,1,0]
test = re_discretize(discrete_values, values_to_discretize)
print(test)

0    1.3
1    3.6
2    3.6
3    1.3
4    1.3
5    1.3
dtype: float64


In [17]:
def re_discretize(discrete_values, values_to_discretize):
    """ This function is useful for re-discretizing the values of
    synthetic individuals after SMOTE in any numerical variable that
    is originally discrete.
    It rounds the values in the array values_to_discretize to the
    closest value in the array discrete_values (which represents the
    truly existing values of the discrete variable)

    Parameters
    ----------
    discrete_values : Array-like
        Original discrete values, sorted in ascending/descending order
    values_to_discretize : Array-like
        Values to be discretized

    Returns
    -------
    np.ndarray
        Same shape as values_to_discretize, rounded to the
        nearest value found in discrete_values
    """
    # The function searchsorted identifies for every value in
    # values_to_discretize the index that should be used if inserting
    # values, to maintain an ordered array
    insertion_index = np.searchsorted(discrete_values, values_to_discretize)

    # We use this index to get the closest lower index and the closest upper
    # index
    index_closest_lower = (insertion_index - 1).clip(min=0)
    index_closest_upper = insertion_index.clip(max=len(discrete_values) - 1)

    # We use these indexes to compute the distances between a value in
    # values_to_discretize and its closest values (lower and upper) in
    # discrete_values
    closest_indexes = np.vstack((index_closest_lower, index_closest_upper))
    closest_values = np.take(discrete_values, closest_indexes)
    shortest_distances = abs(closest_values - values_to_discretize)

    # We then use the value that is the nearest between these lower
    # and upper bounds
    nearest_value = shortest_distances.argmin(axis=0)
    discretized_values = closest_values[
        nearest_value, np.arange(len(values_to_discretize))
    ]
    discretized_values = pd.Series(discretized_values)
    return discretized_values

In [18]:
def Discret_SN_result(df) :
  new_liste_cols = X_train_numeric_colonnes + X_train_categorical_columns
  df = pd.DataFrame(df,columns= new_liste_cols)
  #categorical_columns, discret_columns, float_columns = df_list_colonnes(df)
  for col in X_train_discret_columns :
    df[col] = re_discretize(df[col].unique().tolist(), df[col].tolist())
  return df
Discret_SN_resultstep = FunctionTransformer(Discret_SN_result)




In [1]:
from sklearn.linear_model import LogisticRegression

In [27]:
SN_pipe = Pipeline(
    steps=[
        ('preparation',preparation),
        ('smotenc', sm)        
        ,('discret',Discret_SN_resultstep)
        ,('encode', encode_cat)
        ,('test', LogisticRegression())
    ]
)

In [20]:
SN_pipe

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('quanti',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standard',
                                                                   RobustScaler())]),
                                                  ['index', 'SK_ID_CURR',
                                                   'CNT_CHILDREN', 'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGISTRATION',
                                                   'DAYS_ID_PUBLISH',
                                                   'OWN_CAR_AGE', 'FLAG_MOBIL',
                                                   'FLAG_EMP_PHONE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_CONT_MOBILE...
                                                  ['NAME_CONTRACT_TYPE',
                                                   'CODE_GENDER',
                                                   'FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_TYPE_SUITE',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'WEEKDAY_APPR_PROCESS_START',
                                                   'ORGANIZATION_TYPE',
                                                   'FONDKAPREMONT_MODE',
                                                   'HOUSETYPE_MODE',
                                                   'WALLSMATERIAL_MODE',
                                                   'EMERGENCYSTATE_MODE'])])),
                ('test', LogisticRegression())])

In [21]:
SN_transfo = SN_pipe.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [22]:
SN_transfo

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('quanti',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standard',
                                                                   RobustScaler())]),
                                                  ['index', 'SK_ID_CURR',
                                                   'CNT_CHILDREN', 'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGISTRATION',
                                                   'DAYS_ID_PUBLISH',
                                                   'OWN_CAR_AGE', 'FLAG_MOBIL',
                                                   'FLAG_EMP_PHONE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_CONT_MOBILE...
                                                  ['NAME_CONTRACT_TYPE',
                                                   'CODE_GENDER',
                                                   'FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_TYPE_SUITE',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'WEEKDAY_APPR_PROCESS_START',
                                                   'ORGANIZATION_TYPE',
                                                   'FONDKAPREMONT_MODE',
                                                   'HOUSETYPE_MODE',
                                                   'WALLSMATERIAL_MODE',
                                                   'EMERGENCYSTATE_MODE'])])),
                ('test', LogisticRegression())])

In [24]:

y_pred = SN_transfo.predict(X_train)


In [26]:
y_pred

array([0, 0, 0, ..., 0, 1, 0])

In [28]:
print(len(y_pred))
print(X_train.shape)

61502
(61502, 677)


In [25]:
SN_transfo.score(X_train,y_pred)

1.0

In [17]:
X_stan_for_Smotenc = SN_transfo.transform(X_train)

In [28]:
len(SN_transfo['encode'].get_feature_names_out().tolist())


804

In [29]:
list_cols = pd.Series(SN_transfo['encode'].get_feature_names_out().tolist())
# print(list_cols)
list_cols = list_cols.str.replace('quanti__','').str.replace('remainder__','')
list_cols = list_cols.tolist()
print(list_cols)

['NAME_CONTRACT_TYPE_Cash loans', 'NAME_CONTRACT_TYPE_Revolving loans', 'CODE_GENDER_F', 'CODE_GENDER_M', 'FLAG_OWN_CAR_N', 'FLAG_OWN_CAR_Y', 'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y', 'NAME_TYPE_SUITE_Children', 'NAME_TYPE_SUITE_Family', 'NAME_TYPE_SUITE_Group of people', 'NAME_TYPE_SUITE_Other_A', 'NAME_TYPE_SUITE_Other_B', 'NAME_TYPE_SUITE_Spouse, partner', 'NAME_TYPE_SUITE_Unaccompanied', 'NAME_TYPE_SUITE_manquant', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Academic degree', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher', 'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married',

In [30]:
X_stan_for_Smotenc = pd.DataFrame(X_stan_for_Smotenc,columns= list_cols)

In [31]:
X_stan_for_Smotenc.shape

(61502, 804)

In [48]:
new_liste_cols = X_train_numeric_colonnes + X_train_categorical_columns

In [50]:
X_stan_for_Smotenc = pd.DataFrame(X_stan_for_Smotenc,columns= new_liste_cols)

In [76]:
categorical_columns, discret_columns, float_columns = df_list_colonnes(X_stan_for_Smotenc)
num_col = discret_columns + float_columns
#sm = SMOTENC(categorical_features=[i for i in range(len(num_col),len(X_train_categorical_columns+X_train_numeric_colonnes))], sampling_strategy=0.1, k_neighbors=2)
# X_res_SMOTENC, y_res_SMOTENC = sm.fit_resample(X_stan_for_Smotenc, y_train)

In [83]:
range(661,677)

range(661, 677)

In [77]:
X_res_SMOTENC, y_res_SMOTENC = sm.fit_resample(X_stan_for_Smotenc, y_train)

In [78]:
X_res_SMOTENC.shape

(62190, 677)

In [56]:
X_stan_for_Smotenc.head()

,index,SK_ID_CURR,CNT_CHILDREN,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,0.110819,0.110894,0.0,0.738314,0.751762,0.726789,0.141318,0.0,0.0,0.0,...,Secondary / secondary special,Civil marriage,House / apartment,Laborers,MONDAY,Business Entity Type 3,manquant,block of flats,"Stone, brick",No
1,0.209424,0.209971,0.0,-0.73611,0.498645,-0.802202,-0.485684,0.0,0.0,0.0,...,Higher education,Married,House / apartment,High skill tech staff,TUESDAY,Police,reg oper account,block of flats,Panel,No
2,-0.373928,-0.373339,1.0,0.587303,-0.42439,0.003303,-0.240318,0.0,0.0,0.0,...,Secondary / secondary special,Married,House / apartment,Laborers,FRIDAY,Business Entity Type 2,reg oper account,block of flats,Panel,No
3,-0.64368,-0.644514,0.0,-0.735007,0.785908,0.589174,-0.533825,0.0,0.0,0.0,...,Secondary / secondary special,Civil marriage,House / apartment,manquant,TUESDAY,Self-employed,manquant,manquant,manquant,manquant
4,-0.94431,-0.945144,0.0,-0.716544,-3.604878,0.152294,-0.478307,0.0,0.0,0.0,...,Higher education,Separated,House / apartment,Accountants,MONDAY,Government,reg oper account,block of flats,Panel,No


In [55]:
categorical_columns

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

## <span id="s1" style="font-family:Latin fonts;font-size:1.6em;">Métriques</span><br>
recall / sensibilité et F_score, matrice de confusion F1_score (favoriser à égalité précision et rappel <br>
prendre tout et retravailler le seuil d'affectation ?<br>
[Sommaire](#rt)

# <span id="s2" style="font-family:Latin fonts;font-size:1.8em;"> 2. Classification supervisée</span><br>
[Sommaire](#rt)

# <span id="s3" style="font-family:Latin fonts;font-size:1.8em;"> 3. Fixer les hypers-paramètres</span><br>
[Sommaire](#rt)

# <span id="s4" style="font-family:Latin fonts;font-size:1.8em;"> 4. Choix de l'algorythme</span><br>
[Sommaire](#rt)

In [ ]:
df = pd.read_csv(chemin+'application_train.csv')

In [ ]:
print(df['CODE_GENDER'].value_counts())
print(df['FLAG_OWN_CAR'].value_counts())
print(df['FLAG_OWN_REALTY'].value_counts())

F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64
N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64
Y    213312
N     94199
Name: FLAG_OWN_REALTY, dtype: int64


In [ ]:
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])

In [ ]:
df.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [ ]:



# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv(chemin+'application_train.csv', nrows= num_rows)
    test_df = pd.read_csv(chemin+'application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv(chemin+'bureau.csv', nrows = num_rows)
    bb = pd.read_csv(chemin+'bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv(chemin+'previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv(chemin+'POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg
    
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv(chemin+'installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv(chemin+'credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')


def main(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    #with timer("Run LightGBM with kfold"):
    #    feat_importance = kfold_lightgbm(df, num_folds= 10, stratified= False, debug= debug)
    return df


In [ ]:
submission_file_name = "submission_kernel02.csv"
with timer("Full model run"):
    df = main()
    df.to_csv("df_final.csv", index= False)

Train samples: 307511, test samples: 48744
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 42s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 50s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 26s
Installments payments df shape: (339587, 26)
Process installments payments - done in 62s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 49s
Full model run - done in 459s


In [ ]:
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(df.columns.tolist())

798

In [ ]:
df.columns.tolist()

['index',
 'SK_ID_CURR',
 'TARGET',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMEN

In [ ]:
df["TARGET"].value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [ ]:
test_df = df[df['TARGET'].isnull()]
test_df.to_csv("df_whithout_target.csv", index= False)

In [ ]:
test_df.shape

(48744, 798)

In [ ]:
df_with_target = df[df['TARGET'].notnull()]
df_with_target.to_csv("df_whith_target.csv", index= False)

In [ ]:
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df


In [ ]:
from imblearn.over_sampling import SMOTE

# Utilisation du SMOTE sur les données d'apprentissage
smote = SMOTE(sampling_strategy=0.5)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Liste contenant les indices des variables numériques discrètes
numeric_discrete_col_list = [3, 4]

for numeric_discrete_col in numeric_discrete_col_list:
    X_train_res[:, numeric_discrete_col] = re_discretize(
        np.unique(X_train[:, numeric_discrete_col]),
        X_train_res[:, numeric_discrete_col],
    )

In [ ]:
# Preprocess POS_CASH_balance.csv
def pos_cash_t(num_rows = None, nan_as_category = True):
    pos = pd.read_csv(chemin+'POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

In [ ]:
df_test = pos_cash_t()

In [ ]:
for r in df_test.columns.tolist() :
    print(r)

POS_MONTHS_BALANCE_MAX
POS_MONTHS_BALANCE_MEAN
POS_MONTHS_BALANCE_SIZE
POS_SK_DPD_MAX
POS_SK_DPD_MEAN
POS_SK_DPD_DEF_MAX
POS_SK_DPD_DEF_MEAN
POS_NAME_CONTRACT_STATUS_Active_MEAN
POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN
POS_NAME_CONTRACT_STATUS_Approved_MEAN
POS_NAME_CONTRACT_STATUS_Canceled_MEAN
POS_NAME_CONTRACT_STATUS_Completed_MEAN
POS_NAME_CONTRACT_STATUS_Demand_MEAN
POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN
POS_NAME_CONTRACT_STATUS_Signed_MEAN
POS_NAME_CONTRACT_STATUS_XNA_MEAN
POS_NAME_CONTRACT_STATUS_nan_MEAN
POS_COUNT


In [ ]:
col = 'PREV_NAME_GOODS_CATEGORY_XNA_MEAN'
print(df_test[col].unique())
print(len(df_test[col].unique()))


[0.         0.33333333 0.5        0.77777778 0.66666667 0.4
 0.75       0.16666667 0.625      0.86363636 0.2        0.88888889
 0.25       1.         0.83333333 0.9        0.7826087  0.28571429
 0.6        0.85714286 0.96153846 0.8        0.42857143 0.3
 0.21428571 0.46666667 0.76923077 0.81818182 0.36363636 0.55555556
 0.54545455 0.57142857 0.76470588 0.72727273 0.71428571 0.41176471
 0.78571429 0.30769231 0.88235294 0.6875     0.82608696 0.14285714
 0.73333333 0.94117647 0.875      0.90909091 0.85185185 0.41666667
 0.4375     0.84615385 0.44444444 0.27272727 0.92307692 0.26666667
 0.47368421 0.95454545 0.45454545 0.09090909 0.63636364 0.125
 0.1        0.38461538 0.69230769 0.22222222 0.7        0.18181818
 0.91666667 0.64285714 0.9375     0.92857143 0.58333333 0.82352941
 0.47058824 0.31578947 0.375      0.73913043 0.95652174 0.86666667
 0.95       0.46153846 0.93333333 0.95238095 0.9047619  0.94736842
 0.70588235 0.11111111 0.52631579 0.07692308 0.53846154 0.68421053
 0.73684211 0.

In [ ]:
def bureau_and_balance_test2(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv(chemin+'bureau.csv', nrows = num_rows)
    bb = pd.read_csv(chemin+'bureau_balance.csv', nrows = num_rows)
    bbt, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureaut, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

In [ ]:
df_bureau = bureau_and_balance_test2()

KeyError: "Column(s) ['STATUS_0', 'STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C', 'STATUS_X', 'STATUS_nan'] do not exist"